<img src='./images/charseq.jpeg' width=50% />

This network is based off of Andrej Karpathy's post on RNNs and implementation in Torch. Below is the general architecture of the character-wise RNN.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load data

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

## Tokenization

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
encoded[:100]

array([62, 16, 57, 36, 38, 19, 76, 37, 18, 65, 65, 65, 10, 57, 36, 36, 29,
       37, 51, 57, 48,  8,  0,  8, 19, 13, 37, 57, 76, 19, 37, 57,  0,  0,
       37, 57,  0,  8,  6, 19, 73, 37, 19, 56, 19, 76, 29, 37, 21, 71, 16,
       57, 36, 36, 29, 37, 51, 57, 48,  8,  0, 29, 37,  8, 13, 37, 21, 71,
       16, 57, 36, 36, 29, 37,  8, 71, 37,  8, 38, 13, 37, 39, 40, 71, 65,
       40, 57, 29, 30, 65, 65, 20, 56, 19, 76, 29, 38, 16,  8, 71])

## Preprocess data

In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Make training mini-batches

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[62 16 57 36 38 19 76 37 18 65]
 [13 39 71 37 38 16 57 38 37 57]
 [19 71 28 37 39 76 37 57 37 51]
 [13 37 38 16 19 37 22 16  8 19]
 [37 13 57 40 37 16 19 76 37 38]
 [22 21 13 13  8 39 71 37 57 71]
 [37 52 71 71 57 37 16 57 28 37]
 [33 77  0 39 71 13  6 29 30 37]]

y
 [[16 57 36 38 19 76 37 18 65 65]
 [39 71 37 38 16 57 38 37 57 38]
 [71 28 37 39 76 37 57 37 51 39]
 [37 38 16 19 37 22 16  8 19 51]
 [13 57 40 37 16 19 76 37 38 19]
 [21 13 13  8 39 71 37 57 71 28]
 [52 71 71 57 37 16 57 28 37 13]
 [77  0 39 71 13  6 29 30 37 61]]


## Build the network

In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [13]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


## Train the network

In [14]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [15]:
batch_size = 128
seq_length = 100
n_epochs = 200 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/200... Step: 10... Loss: 3.2514... Val Loss: 3.2327
Epoch: 1/200... Step: 20... Loss: 3.1345... Val Loss: 3.1253
Epoch: 1/200... Step: 30... Loss: 3.1358... Val Loss: 3.1215
Epoch: 1/200... Step: 40... Loss: 3.1152... Val Loss: 3.1223
Epoch: 1/200... Step: 50... Loss: 3.1378... Val Loss: 3.1167
Epoch: 1/200... Step: 60... Loss: 3.1166... Val Loss: 3.1143
Epoch: 1/200... Step: 70... Loss: 3.1062... Val Loss: 3.1121
Epoch: 1/200... Step: 80... Loss: 3.1192... Val Loss: 3.1055
Epoch: 1/200... Step: 90... Loss: 3.1037... Val Loss: 3.0871
Epoch: 1/200... Step: 100... Loss: 3.0599... Val Loss: 3.0484
Epoch: 1/200... Step: 110... Loss: 2.9885... Val Loss: 2.9738
Epoch: 1/200... Step: 120... Loss: 2.8779... Val Loss: 2.8761
Epoch: 1/200... Step: 130... Loss: 2.8304... Val Loss: 2.7980
Epoch: 2/200... Step: 140... Loss: 2.7170... Val Loss: 2.6686
Epoch: 2/200... Step: 150... Loss: 2.6154... Val Loss: 2.5666
Epoch: 2/200... Step: 160... Loss: 2.5425... Val Loss: 2.5039
Epoch: 2/200... S

Epoch: 10/200... Step: 1330... Loss: 1.4003... Val Loss: 1.4200
Epoch: 10/200... Step: 1340... Loss: 1.3902... Val Loss: 1.4186
Epoch: 10/200... Step: 1350... Loss: 1.3770... Val Loss: 1.4185
Epoch: 10/200... Step: 1360... Loss: 1.3940... Val Loss: 1.4181
Epoch: 10/200... Step: 1370... Loss: 1.3774... Val Loss: 1.4169
Epoch: 10/200... Step: 1380... Loss: 1.4232... Val Loss: 1.4108
Epoch: 10/200... Step: 1390... Loss: 1.4253... Val Loss: 1.4096
Epoch: 11/200... Step: 1400... Loss: 1.4226... Val Loss: 1.4105
Epoch: 11/200... Step: 1410... Loss: 1.4384... Val Loss: 1.4074
Epoch: 11/200... Step: 1420... Loss: 1.4311... Val Loss: 1.4008
Epoch: 11/200... Step: 1430... Loss: 1.3901... Val Loss: 1.4039
Epoch: 11/200... Step: 1440... Loss: 1.4253... Val Loss: 1.4034
Epoch: 11/200... Step: 1450... Loss: 1.3479... Val Loss: 1.4008
Epoch: 11/200... Step: 1460... Loss: 1.3769... Val Loss: 1.4027
Epoch: 11/200... Step: 1470... Loss: 1.3757... Val Loss: 1.4008
Epoch: 11/200... Step: 1480... Loss: 1.3

Epoch: 19/200... Step: 2620... Loss: 1.2248... Val Loss: 1.3032
Epoch: 19/200... Step: 2630... Loss: 1.2311... Val Loss: 1.3040
Epoch: 19/200... Step: 2640... Loss: 1.2428... Val Loss: 1.3039
Epoch: 20/200... Step: 2650... Loss: 1.2573... Val Loss: 1.2999
Epoch: 20/200... Step: 2660... Loss: 1.2559... Val Loss: 1.3041
Epoch: 20/200... Step: 2670... Loss: 1.2522... Val Loss: 1.3005
Epoch: 20/200... Step: 2680... Loss: 1.2582... Val Loss: 1.2997
Epoch: 20/200... Step: 2690... Loss: 1.2419... Val Loss: 1.3010
Epoch: 20/200... Step: 2700... Loss: 1.2453... Val Loss: 1.2979
Epoch: 20/200... Step: 2710... Loss: 1.2169... Val Loss: 1.2997
Epoch: 20/200... Step: 2720... Loss: 1.2220... Val Loss: 1.2986
Epoch: 20/200... Step: 2730... Loss: 1.2175... Val Loss: 1.2962
Epoch: 20/200... Step: 2740... Loss: 1.2098... Val Loss: 1.2983
Epoch: 20/200... Step: 2750... Loss: 1.2270... Val Loss: 1.2985
Epoch: 20/200... Step: 2760... Loss: 1.2249... Val Loss: 1.2969
Epoch: 20/200... Step: 2770... Loss: 1.2

Epoch: 29/200... Step: 3910... Loss: 1.1508... Val Loss: 1.2429
Epoch: 29/200... Step: 3920... Loss: 1.1628... Val Loss: 1.2498
Epoch: 29/200... Step: 3930... Loss: 1.1724... Val Loss: 1.2494
Epoch: 29/200... Step: 3940... Loss: 1.1345... Val Loss: 1.2462
Epoch: 29/200... Step: 3950... Loss: 1.1463... Val Loss: 1.2487
Epoch: 29/200... Step: 3960... Loss: 1.1460... Val Loss: 1.2441
Epoch: 29/200... Step: 3970... Loss: 1.1767... Val Loss: 1.2478
Epoch: 29/200... Step: 3980... Loss: 1.1438... Val Loss: 1.2475
Epoch: 29/200... Step: 3990... Loss: 1.1325... Val Loss: 1.2460
Epoch: 29/200... Step: 4000... Loss: 1.1499... Val Loss: 1.2423
Epoch: 29/200... Step: 4010... Loss: 1.1248... Val Loss: 1.2447
Epoch: 29/200... Step: 4020... Loss: 1.1323... Val Loss: 1.2405
Epoch: 29/200... Step: 4030... Loss: 1.1542... Val Loss: 1.2448
Epoch: 30/200... Step: 4040... Loss: 1.1471... Val Loss: 1.2522
Epoch: 30/200... Step: 4050... Loss: 1.1497... Val Loss: 1.2453
Epoch: 30/200... Step: 4060... Loss: 1.1

Epoch: 38/200... Step: 5200... Loss: 1.0947... Val Loss: 1.2340
Epoch: 38/200... Step: 5210... Loss: 1.1018... Val Loss: 1.2336
Epoch: 38/200... Step: 5220... Loss: 1.0896... Val Loss: 1.2389
Epoch: 38/200... Step: 5230... Loss: 1.0754... Val Loss: 1.2404
Epoch: 38/200... Step: 5240... Loss: 1.1008... Val Loss: 1.2385
Epoch: 38/200... Step: 5250... Loss: 1.0934... Val Loss: 1.2360
Epoch: 38/200... Step: 5260... Loss: 1.0951... Val Loss: 1.2383
Epoch: 38/200... Step: 5270... Loss: 1.0784... Val Loss: 1.2324
Epoch: 38/200... Step: 5280... Loss: 1.0827... Val Loss: 1.2275
Epoch: 39/200... Step: 5290... Loss: 1.0861... Val Loss: 1.2410
Epoch: 39/200... Step: 5300... Loss: 1.1041... Val Loss: 1.2362
Epoch: 39/200... Step: 5310... Loss: 1.1050... Val Loss: 1.2322
Epoch: 39/200... Step: 5320... Loss: 1.1116... Val Loss: 1.2319
Epoch: 39/200... Step: 5330... Loss: 1.0806... Val Loss: 1.2289
Epoch: 39/200... Step: 5340... Loss: 1.0991... Val Loss: 1.2337
Epoch: 39/200... Step: 5350... Loss: 1.0

...

Epoch: 156/200... Step: 21650... Loss: 0.8843... Val Loss: 1.3939
Epoch: 156/200... Step: 21660... Loss: 0.8583... Val Loss: 1.4027
Epoch: 156/200... Step: 21670... Loss: 0.8378... Val Loss: 1.4210
Epoch: 156/200... Step: 21680... Loss: 0.8722... Val Loss: 1.3975
Epoch: 157/200... Step: 21690... Loss: 0.8748... Val Loss: 1.3954
Epoch: 157/200... Step: 21700... Loss: 0.8687... Val Loss: 1.3985
Epoch: 157/200... Step: 21710... Loss: 0.8720... Val Loss: 1.4053
Epoch: 157/200... Step: 21720... Loss: 0.8774... Val Loss: 1.3960
Epoch: 157/200... Step: 21730... Loss: 0.8634... Val Loss: 1.4144
Epoch: 157/200... Step: 21740... Loss: 0.8784... Val Loss: 1.4037
Epoch: 157/200... Step: 21750... Loss: 0.8872... Val Loss: 1.3957
Epoch: 157/200... Step: 21760... Loss: 0.8613... Val Loss: 1.3867
Epoch: 157/200... Step: 21770... Loss: 0.8725... Val Loss: 1.4068
Epoch: 157/200... Step: 21780... Loss: 0.8638... Val Loss: 1.4017
Epoch: 157/200... Step: 21790... Loss: 0.8485... Val Loss: 1.3955
Epoch: 157

Epoch: 165/200... Step: 22900... Loss: 0.8501... Val Loss: 1.3998
Epoch: 165/200... Step: 22910... Loss: 0.8517... Val Loss: 1.4136
Epoch: 165/200... Step: 22920... Loss: 0.8661... Val Loss: 1.4232
Epoch: 165/200... Step: 22930... Loss: 0.8613... Val Loss: 1.4092
Epoch: 166/200... Step: 22940... Loss: 0.8784... Val Loss: 1.4075
Epoch: 166/200... Step: 22950... Loss: 0.8663... Val Loss: 1.4216
Epoch: 166/200... Step: 22960... Loss: 0.8543... Val Loss: 1.4052
Epoch: 166/200... Step: 22970... Loss: 0.8678... Val Loss: 1.4038
Epoch: 166/200... Step: 22980... Loss: 0.8489... Val Loss: 1.4117
Epoch: 166/200... Step: 22990... Loss: 0.8794... Val Loss: 1.4153
Epoch: 166/200... Step: 23000... Loss: 0.8807... Val Loss: 1.4081
Epoch: 166/200... Step: 23010... Loss: 0.8647... Val Loss: 1.4022
Epoch: 166/200... Step: 23020... Loss: 0.8563... Val Loss: 1.4081
Epoch: 166/200... Step: 23030... Loss: 0.8588... Val Loss: 1.4131
Epoch: 166/200... Step: 23040... Loss: 0.8807... Val Loss: 1.4010
Epoch: 166

Epoch: 174/200... Step: 24150... Loss: 0.8612... Val Loss: 1.4186
Epoch: 174/200... Step: 24160... Loss: 0.8429... Val Loss: 1.4216
Epoch: 174/200... Step: 24170... Loss: 0.8369... Val Loss: 1.4226
Epoch: 174/200... Step: 24180... Loss: 0.8601... Val Loss: 1.4178
Epoch: 175/200... Step: 24190... Loss: 0.8609... Val Loss: 1.4122
Epoch: 175/200... Step: 24200... Loss: 0.8607... Val Loss: 1.4389
Epoch: 175/200... Step: 24210... Loss: 0.8476... Val Loss: 1.4026
Epoch: 175/200... Step: 24220... Loss: 0.8477... Val Loss: 1.4089
Epoch: 175/200... Step: 24230... Loss: 0.8583... Val Loss: 1.4249
Epoch: 175/200... Step: 24240... Loss: 0.8405... Val Loss: 1.4191
Epoch: 175/200... Step: 24250... Loss: 0.8626... Val Loss: 1.4221
Epoch: 175/200... Step: 24260... Loss: 0.8678... Val Loss: 1.4110
Epoch: 175/200... Step: 24270... Loss: 0.8561... Val Loss: 1.4217
Epoch: 175/200... Step: 24280... Loss: 0.8588... Val Loss: 1.4316
Epoch: 175/200... Step: 24290... Loss: 0.8511... Val Loss: 1.4116
Epoch: 175

Epoch: 183/200... Step: 25400... Loss: 0.8691... Val Loss: 1.4191
Epoch: 183/200... Step: 25410... Loss: 0.8494... Val Loss: 1.4279
Epoch: 183/200... Step: 25420... Loss: 0.8291... Val Loss: 1.4392
Epoch: 183/200... Step: 25430... Loss: 0.8564... Val Loss: 1.4213
Epoch: 184/200... Step: 25440... Loss: 0.8463... Val Loss: 1.4181
Epoch: 184/200... Step: 25450... Loss: 0.8426... Val Loss: 1.4289
Epoch: 184/200... Step: 25460... Loss: 0.8653... Val Loss: 1.4056
Epoch: 184/200... Step: 25470... Loss: 0.8404... Val Loss: 1.4332
Epoch: 184/200... Step: 25480... Loss: 0.8180... Val Loss: 1.4287
Epoch: 184/200... Step: 25490... Loss: 0.8575... Val Loss: 1.4172
Epoch: 184/200... Step: 25500... Loss: 0.8651... Val Loss: 1.4290
Epoch: 184/200... Step: 25510... Loss: 0.8681... Val Loss: 1.4267
Epoch: 184/200... Step: 25520... Loss: 0.8605... Val Loss: 1.4273
Epoch: 184/200... Step: 25530... Loss: 0.8421... Val Loss: 1.4369
Epoch: 184/200... Step: 25540... Loss: 0.8470... Val Loss: 1.4233
Epoch: 184

Epoch: 192/200... Step: 26650... Loss: 0.8485... Val Loss: 1.4450
Epoch: 192/200... Step: 26660... Loss: 0.8386... Val Loss: 1.4337
Epoch: 192/200... Step: 26670... Loss: 0.8293... Val Loss: 1.4386
Epoch: 192/200... Step: 26680... Loss: 0.8478... Val Loss: 1.4230
Epoch: 193/200... Step: 26690... Loss: 0.8358... Val Loss: 1.4367
Epoch: 193/200... Step: 26700... Loss: 0.8505... Val Loss: 1.4401
Epoch: 193/200... Step: 26710... Loss: 0.8460... Val Loss: 1.4201
Epoch: 193/200... Step: 26720... Loss: 0.8439... Val Loss: 1.4379
Epoch: 193/200... Step: 26730... Loss: 0.8286... Val Loss: 1.4231
Epoch: 193/200... Step: 26740... Loss: 0.8300... Val Loss: 1.4320
Epoch: 193/200... Step: 26750... Loss: 0.8582... Val Loss: 1.4438
Epoch: 193/200... Step: 26760... Loss: 0.8366... Val Loss: 1.4206
Epoch: 193/200... Step: 26770... Loss: 0.8215... Val Loss: 1.4220
Epoch: 193/200... Step: 26780... Loss: 0.8253... Val Loss: 1.4434
Epoch: 193/200... Step: 26790... Loss: 0.8573... Val Loss: 1.4367
Epoch: 193

## Checkpoint

In [16]:
# change the name, for saving multiple files
model_name = f'weights/rnn_{n_epochs}_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

## Predict

In [17]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [18]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [19]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna and Levin,
as she studiously recalled the holy picture, and tortured the
beillish bright on her hand, and looked straight into his face when the
second district council who had come over his fure.

"Well, I was mad trumaly by a sign of his room; and then, I can't give it out
about her wife, and nothing must be in a both fore. A delightful
strength too.... The children had been turned up. He dressed, the
sick man was a matter of something in my house. I am a misery from
my position. I have always pleased to get out of time to say..."

"I have never had the contrary, I want to talk to you," said Stepan
Arkadyevitch, smiling, smiling, as he listened to him in spite of him an
intervated soup and smiled. There was no sort of composure with some
pictures.

Anna had brought themselves away by a desire to sport about him to
bed the drawing room the steps were stood one and the last act of
spiritual society.

Stepan Arkadyevitch's finger was never a little house and carried him to the
sens

In [20]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open(model_name, 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [21]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said
that the marshal of the province so long aforh was such an heart. But he
was ashamed for the dinner his former danging ones. A clerk who came out
to the princess's suffering, she heard a little of the box she called him a
follow that had been parted of worries were at action by his father's
father. Then his face was like a man who had been taken it every minute.
She saw that something was wrong with her, and he changed the window
and clinging to the corridor.

"You certainly do nothing?" said Stepan Arkadyevitch.

"I don't know him, you want to keep her today," he said, smiling, and
pressing her eyes, his eyes swinging her hands and sangs the cold and rather
face with a certain tears cut her to the boy.

She stopped something with his short and glancing lips of the
carriage.

"Well, and what's the man taken about?" he asked.

"Well, then a moment, the man had come to make a place in me, who
consists an interval, but take the carriages and lady would be
nurse."

"I don't 

---